# **RA1: quarterly panel data 2**

Dong Gyun Ko <br/>
last updated: october 11, 2022 <br/>

In [1]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader as pdr
import pandas_datareader.data as web
from pandas_datareader import wb
from pandas_datareader.data import DataReader
import requests # python 3.6

import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# set the working directory

os.chdir('/content/gdrive/MyDrive/Colab Notebooks/RA1_data')

## **1. IMF**

* [IMF](https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854): international monetary fund

### **1.1. python code** <br/>

In [4]:
# import the raw data

df_imf_q_4 = pd.read_csv('df_imf_q_4.csv').sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [5]:
# sort the raw data

df_imf_q = df_imf_q_4.sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [6]:
# filter

cond1 = (df_imf_q['country'] == 'Albania')
cond2 = (df_imf_q['country'] == 'Costa Rica')
cond3 = (df_imf_q['country'] == 'Ecuador')
cond4 = (df_imf_q['country'] == 'Euro Area')
cond5 = (df_imf_q['country'] == 'Honduras')
cond6 = (df_imf_q['country'] == 'Jordan')
cond7 = (df_imf_q['country'] == 'Moldova, Rep. of')
cond8 = (df_imf_q['country'] == 'Montenegro')
cond9 = (df_imf_q['country'] == 'Philippines')
cond10 = (df_imf_q['country'] == 'Serbia, Rep. of')

df_imf_q = df_imf_q.loc[~cond1 & ~cond2 & ~cond3 & ~cond4 & ~cond5 & ~cond6 & ~cond7 & ~cond8 & ~cond9 & ~cond10]

In [7]:
# replace the country

df_imf_q = df_imf_q.replace({'country':'China, P.R.: Hong Kong'}, 'Hong Kong SAR, China')
df_imf_q = df_imf_q.replace({'country':'Croatia, Rep. of'}, 'Croatia')
df_imf_q = df_imf_q.replace({'country':'Czech Rep.'}, 'Czech Republic')
df_imf_q = df_imf_q.replace({'country':'Estonia, Rep. of'}, 'Estonia')
df_imf_q = df_imf_q.replace({'country':'Korea, Rep. of'}, 'Korea, Rep.')
df_imf_q = df_imf_q.replace({'country':'Netherlands, The'}, 'Netherlands')
df_imf_q = df_imf_q.replace({'country':'Poland, Rep. of'}, 'Poland')
df_imf_q = df_imf_q.replace({'country':'Slovak Rep.'}, 'Slovak Republic')
df_imf_q = df_imf_q.replace({'country':'Slovenia, Rep. of'}, 'Slovenia')
df_imf_q = df_imf_q.replace({'country':'Turkey'}, 'Turkiye')

In [8]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_imf_q = df_imf_q[['country', 'quarter', 'imf_q_ngdp', 'imf_q_exp', 'imf_q_imp', 'imf_q_pcon']]
df_imf_q = pd.merge(df_wb_isocode, df_imf_q, how='right', on=['country'])
df_imf_q = df_imf_q.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)
df_imf_q['quarter'] = pd.to_datetime(df_imf_q['quarter'])
df_imf_q['quarter'] = df_imf_q['quarter'].dt.to_period('Q')

In [9]:
# imf quarterly dataset

df_imf_q['imf_q_tradeopen'] = ((df_imf_q['imf_q_exp'] + df_imf_q['imf_q_imp']) / df_imf_q['imf_q_ngdp']) * 100
df_imf_q = df_imf_q.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_imf_q

,country,isocode,quarter,imf_q_ngdp,imf_q_exp,imf_q_imp,imf_q_pcon,imf_q_tradeopen
0,Argentina,ARG,2004Q1,1.148120e+11,2.720770e+10,1.840540e+10,7.530930e+10,39.728513
1,Argentina,ARG,2004Q2,1.210270e+11,2.847930e+10,1.953720e+10,7.629520e+10,39.674205
2,Argentina,ARG,2004Q3,1.224820e+11,2.940970e+10,2.092230e+10,7.933080e+10,41.093385
3,Argentina,ARG,2004Q4,1.267940e+11,3.059170e+10,2.285290e+10,8.114660e+10,42.150733
4,Argentina,ARG,2005Q1,1.335390e+11,3.155180e+10,2.336490e+10,8.039200e+10,41.124091
...,...,...,...,...,...,...,...,...
6502,South Africa,ZAF,2021Q2,1.562120e+12,5.028860e+11,3.773610e+11,7.524750e+11,56.349512
6503,South Africa,ZAF,2021Q3,1.546850e+12,4.727160e+11,3.795390e+11,7.317760e+11,55.096163
6504,South Africa,ZAF,2021Q4,1.571830e+12,4.953190e+11,4.280150e+11,7.536310e+11,58.742612
6505,South Africa,ZAF,2022Q1,1.608260e+12,5.400740e+11,4.669890e+11,7.628920e+11,62.618171


### **1.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|gross domestic product, nominal|imf_q_ngdp|DCU|quarterly|NGDP_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|2|export of goods and services, nominal|imf_q_exp|DCU|quarterly|NX_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|3|import of goods and services, nominal|imf_q_imp|DCU|quarterly|NM_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|4|trade openness, (export + import) / GDP|imf_q_tradeopen|% of GDP|quarterly|-|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|5|private sector final consumption expenditure, real|imf_q_pcon|DCU|quarterly|NCP_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|

### **1.3. data summary** <br/>

In [10]:
# number of countries & sample period

print('number of countries:', df_imf_q['isocode'].unique().shape[0])
print('start:', np.min(df_imf_q['quarter'].unique()))
print('end:', np.max(df_imf_q['quarter'].unique()))

number of countries: 50
start: 1950Q1
end: 2022Q2


In [11]:
# non-null count by each variables

df_imf_q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6507 entries, 0 to 6506
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype        
---  ------           --------------  -----        
 0   country          6507 non-null   object       
 1   isocode          6507 non-null   object       
 2   quarter          6507 non-null   period[Q-DEC]
 3   imf_q_ngdp       6191 non-null   float64      
 4   imf_q_exp        6191 non-null   float64      
 5   imf_q_imp        6191 non-null   float64      
 6   imf_q_pcon       6131 non-null   float64      
 7   imf_q_tradeopen  6191 non-null   float64      
dtypes: float64(5), object(2), period[Q-DEC](1)
memory usage: 406.8+ KB


In [12]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_imf_q.describe(), 2)

,imf_q_ngdp,imf_q_exp,imf_q_imp,imf_q_pcon,imf_q_tradeopen
count,6191.00,6191.00,6191.00,6131.00,6191.00
mean,42882903097010.86,10665512767181.47,10132811503065.73,23008212211023.31,81.89
std,297459619286877.69,66369216310031.29,63027889369226.33,134000145097356.73,55.69
min,267985000.00,94170000.00,62187000.00,813515856.40,7.55
25%,42382768350.50,13782150000.00,14460708446.50,29401150000.00,47.12
50%,204178000000.00,68128500000.00,65993000000.00,153786000000.00,66.17
75%,896335000000.00,292429500000.00,277726000000.00,476171500000.00,102.90
max,4900000000000000.00,1210000000000000.00,1000000000000000.00,1600000000000000.00,395.76


## **2. WB**

* [WB](https://www.worldbank.org/en/research/brief/fiscal-space): world bank
* [README](https://documents1.worldbank.org/curated/en/601211501678994591/pdf/WPS8157.pdf): cross-country database of fiscal space <br/>

### **2.1. python code** <br/>

In [13]:
# generate the q1 dataset

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# import the raw data
df_wb_a = pd.read_csv('df_wb_a_cds_5y.csv').set_index('isocode').transpose()
df_wb_a = pd.DataFrame(df_wb_a.stack(level='isocode')).reset_index()
df_wb_a = df_wb_a.rename(columns={'level_0':'year', 0:'wb_a_cds_5y'})
df_wb_a = df_wb_a[['isocode', 'year', 'wb_a_cds_5y']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_wb_a['year'] = pd.DataFrame(df_wb_a['year'].apply(ymd_q1))
df_wb_a['year'] = pd.to_datetime(df_wb_a['year'])
df_wb_a['year'] = df_wb_a['year'].dt.to_period('Q')
df_wb_a = df_wb_a.rename(columns={'year':'quarter'})
df_wb_a_q1 = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [14]:
# generate the q2 dataset

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# import the raw data
df_wb_a = pd.read_csv('df_wb_a_cds_5y.csv').set_index('isocode').transpose()
df_wb_a = pd.DataFrame(df_wb_a.stack(level='isocode')).reset_index()
df_wb_a = df_wb_a.rename(columns={'level_0':'year', 0:'wb_a_cds_5y'})
df_wb_a = df_wb_a[['isocode', 'year', 'wb_a_cds_5y']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_wb_a['year'] = pd.DataFrame(df_wb_a['year'].apply(ymd_q2))
df_wb_a['year'] = pd.to_datetime(df_wb_a['year'])
df_wb_a['year'] = df_wb_a['year'].dt.to_period('Q')
df_wb_a = df_wb_a.rename(columns={'year':'quarter'})
df_wb_a_q2 = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [15]:
# generate the q3 dataset

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# import the raw data
df_wb_a = pd.read_csv('df_wb_a_cds_5y.csv').set_index('isocode').transpose()
df_wb_a = pd.DataFrame(df_wb_a.stack(level='isocode')).reset_index()
df_wb_a = df_wb_a.rename(columns={'level_0':'year', 0:'wb_a_cds_5y'})
df_wb_a = df_wb_a[['isocode', 'year', 'wb_a_cds_5y']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_wb_a['year'] = pd.DataFrame(df_wb_a['year'].apply(ymd_q3))
df_wb_a['year'] = pd.to_datetime(df_wb_a['year'])
df_wb_a['year'] = df_wb_a['year'].dt.to_period('Q')
df_wb_a = df_wb_a.rename(columns={'year':'quarter'})
df_wb_a_q3 = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [16]:
# generate the q4 dataset

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-10-01'

# import the raw data
df_wb_a = pd.read_csv('df_wb_a_cds_5y.csv').set_index('isocode').transpose()
df_wb_a = pd.DataFrame(df_wb_a.stack(level='isocode')).reset_index()
df_wb_a = df_wb_a.rename(columns={'level_0':'year', 0:'wb_a_cds_5y'})
df_wb_a = df_wb_a[['isocode', 'year', 'wb_a_cds_5y']].sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_wb_a['year'] = pd.DataFrame(df_wb_a['year'].apply(ymd_q4))
df_wb_a['year'] = pd.to_datetime(df_wb_a['year'])
df_wb_a['year'] = df_wb_a['year'].dt.to_period('Q')
df_wb_a = df_wb_a.rename(columns={'year':'quarter'})
df_wb_a_q4 = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [17]:
# merge the raw data

df_wb_a = pd.merge(df_wb_a_q1, df_wb_a_q2, how='outer', on=['isocode', 'quarter', 'wb_a_cds_5y'])
df_wb_a = pd.merge(df_wb_a, df_wb_a_q3, how='outer', on=['isocode', 'quarter', 'wb_a_cds_5y'])
df_wb_a = pd.merge(df_wb_a, df_wb_a_q4, how='outer', on=['isocode', 'quarter', 'wb_a_cds_5y'])

In [18]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_wb_a = pd.merge(df_wb_isocode, df_wb_a, how='right', on=['isocode'])
df_wb_a = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [19]:
# sort the data by year

df_wb_a['year'] = df_wb_a['quarter'].dt.year
df_wb_a = df_wb_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_wb_a = df_wb_a.drop(columns=['year']).reset_index(drop=True)

In [20]:
# select the T and N

# select the T
cond_T = (df_wb_a['quarter'].dt.year >= 1950)

df_ci_a = df_wb_a.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 'CZE',
                          'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN',
                          'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD',
                          'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SVN', 'SWE', 'THA', 'TUR',
                          'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_wb_a['isocode'] == isocode_array[i])


df_wb_a = df_wb_a.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

In [21]:
# annual wb dataset

df_wb_a = df_wb_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_wb_a

,country,isocode,quarter,wb_a_cds_5y
0,Argentina,ARG,1997Q1,373.62
1,Argentina,ARG,1997Q2,373.62
2,Argentina,ARG,1997Q3,373.62
3,Argentina,ARG,1997Q4,373.62
4,Argentina,ARG,1998Q1,609.10
...,...,...,...,...
3199,South Africa,ZAF,2020Q4,289.71
3200,South Africa,ZAF,2021Q1,211.64
3201,South Africa,ZAF,2021Q2,211.64
3202,South Africa,ZAF,2021Q3,211.64


### **2.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|5-year sovereign CDS spreads, basis points|wb_a_cds_5y|basis point|annual|-|-|-|https://www.worldbank.org/en/research/brief/fiscal-space|

### **2.3. data summary** <br/>

In [22]:
# number of countries & sample period

print('number of countries:', df_wb_a['isocode'].unique().shape[0]) # CAN, IND, LUX, MLT, SGP: not exist
print('start:', np.min(df_wb_a['quarter'].unique()))
print('end:', np.max(df_wb_a['quarter'].unique()))

number of countries: 45
start: 1997Q1
end: 2021Q4


In [23]:
# non-null count by each variables

df_wb_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3204 entries, 0 to 3203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype        
---  ------       --------------  -----        
 0   country      3204 non-null   object       
 1   isocode      3204 non-null   object       
 2   quarter      3204 non-null   period[Q-DEC]
 3   wb_a_cds_5y  3204 non-null   float64      
dtypes: float64(1), object(2), period[Q-DEC](1)
memory usage: 100.2+ KB


In [24]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_wb_a.describe(), 2)

,wb_a_cds_5y
count,3204.00
mean,318.80
std,2102.32
min,1.77
25%,34.60
50%,81.62
75%,189.79
max,50990.61


## **3. CI**

* [CI](https://web.pdx.edu/~ito/Chinn-Ito_website.htm): Menzie Chinn, Hero Ito

### **3.1. python code** <br/>

In [25]:
# generate the q1 dataset

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# import the raw data
df_ci_a = pd.read_csv('df_ci_a_kaopen.csv')
df_ci_a = df_ci_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_ci_a['year'] = pd.DataFrame(df_ci_a['year'].apply(ymd_q1))
df_ci_a['year'] = pd.to_datetime(df_ci_a['year'])
df_ci_a['year'] = df_ci_a['year'].dt.to_period('Q')
df_ci_a = df_ci_a.rename(columns={'year':'quarter'})
df_ci_a_q1 = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [26]:
# generate the q2 dataset

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# import the raw data
df_ci_a = pd.read_csv('df_ci_a_kaopen.csv')
df_ci_a = df_ci_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_ci_a['year'] = pd.DataFrame(df_ci_a['year'].apply(ymd_q2))
df_ci_a['year'] = pd.to_datetime(df_ci_a['year'])
df_ci_a['year'] = df_ci_a['year'].dt.to_period('Q')
df_ci_a = df_ci_a.rename(columns={'year':'quarter'})
df_ci_a_q2 = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [27]:
# generate the q3 dataset

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# import the raw data
df_ci_a = pd.read_csv('df_ci_a_kaopen.csv')
df_ci_a = df_ci_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_ci_a['year'] = pd.DataFrame(df_ci_a['year'].apply(ymd_q3))
df_ci_a['year'] = pd.to_datetime(df_ci_a['year'])
df_ci_a['year'] = df_ci_a['year'].dt.to_period('Q')
df_ci_a = df_ci_a.rename(columns={'year':'quarter'})
df_ci_a_q3 = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [28]:
# generate the q4 dataset

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-10-01'

# import the raw data
df_ci_a = pd.read_csv('df_ci_a_kaopen.csv')
df_ci_a = df_ci_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)

# transform year into quarter
df_ci_a['year'] = pd.DataFrame(df_ci_a['year'].apply(ymd_q4))
df_ci_a['year'] = pd.to_datetime(df_ci_a['year'])
df_ci_a['year'] = df_ci_a['year'].dt.to_period('Q')
df_ci_a = df_ci_a.rename(columns={'year':'quarter'})
df_ci_a_q4 = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [29]:
# merge the raw data

df_ci_a = pd.merge(df_ci_a_q1, df_ci_a_q2, how='outer', on=['country', 'isocode', 'quarter', 'ci_a_kaopen', 'ci_a_kaopen_n'])
df_ci_a = pd.merge(df_ci_a, df_ci_a_q3, how='outer', on=['country', 'isocode', 'quarter', 'ci_a_kaopen', 'ci_a_kaopen_n'])
df_ci_a = pd.merge(df_ci_a, df_ci_a_q4, how='outer', on=['country', 'isocode', 'quarter', 'ci_a_kaopen', 'ci_a_kaopen_n'])

df_ci_a = df_ci_a.drop(columns=['country']).reset_index(drop=True)

In [30]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_ci_a = pd.merge(df_wb_isocode, df_ci_a, how='right', on=['isocode'])
df_ci_a = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [31]:
# sort the data by year

df_ci_a['year'] = df_ci_a['quarter'].dt.year
df_ci_a = df_ci_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_ci_a = df_ci_a.drop(columns=['year']).reset_index(drop=True)

In [32]:
# select the T and N

# select the T
cond_T = (df_ci_a['quarter'].dt.year >= 1950)

df_ci_a = df_ci_a.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 'CZE',
                          'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN',
                          'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD',
                          'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SVN', 'SWE', 'THA', 'TUR',
                          'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_ci_a['isocode'] == isocode_array[i])


df_ci_a = df_ci_a.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

In [33]:
# annual ci dataset

df_ci_a = df_ci_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_ci_a

,country,isocode,quarter,ci_a_kaopen,ci_a_kaopen_n
0,Argentina,ARG,1970Q1,1.00,1.00
1,Argentina,ARG,1970Q2,1.00,1.00
2,Argentina,ARG,1970Q3,1.00,1.00
3,Argentina,ARG,1970Q4,1.00,1.00
4,Argentina,ARG,1971Q1,-1.00,0.00
...,...,...,...,...,...
9787,South Africa,ZAF,2019Q4,-1.00,0.00
9788,South Africa,ZAF,2020Q1,-1.00,0.00
9789,South Africa,ZAF,2020Q2,-1.00,0.00
9790,South Africa,ZAF,2020Q3,-1.00,0.00


### **3.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|Chinn-Ito financial openness index|ci_a_kaopen|index|annual|-|-|-|https://web.pdx.edu/~ito/Chinn-Ito_website.htm|
|2|Chinn-Ito financial openness index, normalized|ci_a_kaopen_n|index|annual|-|-|normalzied (0-1)|https://web.pdx.edu/~ito/Chinn-Ito_website.htm|

### **3.3. data summary** <br/>

In [34]:
# number of countries & sample period

print('number of countries:', df_ci_a['isocode'].unique().shape[0]) # LUX, ROU: not exist
print('start:', np.min(df_ci_a['quarter'].unique()))
print('end:', np.max(df_ci_a['quarter'].unique()))

number of countries: 48
start: 1970Q1
end: 2020Q4


In [35]:
# non-null count by each variables

df_ci_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9792 entries, 0 to 9791
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype        
---  ------         --------------  -----        
 0   country        9792 non-null   object       
 1   isocode        9792 non-null   object       
 2   quarter        9792 non-null   period[Q-DEC]
 3   ci_a_kaopen    8600 non-null   float64      
 4   ci_a_kaopen_n  8600 non-null   float64      
dtypes: float64(2), object(2), period[Q-DEC](1)
memory usage: 382.6+ KB


In [36]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_ci_a.describe(), 2)

,ci_a_kaopen,ci_a_kaopen_n
count,8600.00,8600.00
mean,0.76,0.60
std,1.35,0.49
min,-2.00,0.00
25%,0.00,0.00
50%,1.00,1.00
75%,2.00,1.00
max,2.00,1.00


## **4. EWN**

* [EWN](https://www.brookings.edu/research/the-external-wealth-of-nations-database/): the external wealth of nations database <br/>

* Philip R. Lane, Gian Maria Milesi-Ferretti <br/>

### **4.1. python code** <br/>

In [37]:
# generate the q1 dataset

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# import the raw data
df_ewn_a = pd.read_csv('df_ewn_a_iip.csv')
df_ewn_a = df_ewn_a.sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
df_ewn_a['year'] = pd.DataFrame(df_ewn_a['year'].apply(ymd_q1))
df_ewn_a['year'] = pd.to_datetime(df_ewn_a['year'])
df_ewn_a['year'] = df_ewn_a['year'].dt.to_period('Q')
df_ewn_a = df_ewn_a.rename(columns={'year':'quarter'})
df_ewn_a_q1 = df_ewn_a.sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [38]:
# generate the q2 dataset

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# import the raw data
df_ewn_a = pd.read_csv('df_ewn_a_iip.csv')
df_ewn_a = df_ewn_a.sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
df_ewn_a['year'] = pd.DataFrame(df_ewn_a['year'].apply(ymd_q2))
df_ewn_a['year'] = pd.to_datetime(df_ewn_a['year'])
df_ewn_a['year'] = df_ewn_a['year'].dt.to_period('Q')
df_ewn_a = df_ewn_a.rename(columns={'year':'quarter'})
df_ewn_a_q2 = df_ewn_a.sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [39]:
# generate the q3 dataset

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# import the raw data
df_ewn_a = pd.read_csv('df_ewn_a_iip.csv')
df_ewn_a = df_ewn_a.sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
df_ewn_a['year'] = pd.DataFrame(df_ewn_a['year'].apply(ymd_q3))
df_ewn_a['year'] = pd.to_datetime(df_ewn_a['year'])
df_ewn_a['year'] = df_ewn_a['year'].dt.to_period('Q')
df_ewn_a = df_ewn_a.rename(columns={'year':'quarter'})
df_ewn_a_q3 = df_ewn_a.sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [40]:
# generate the q4 dataset

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-10-01'

# import the raw data
df_ewn_a = pd.read_csv('df_ewn_a_iip.csv')
df_ewn_a = df_ewn_a.sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
df_ewn_a['year'] = pd.DataFrame(df_ewn_a['year'].apply(ymd_q4))
df_ewn_a['year'] = pd.to_datetime(df_ewn_a['year'])
df_ewn_a['year'] = df_ewn_a['year'].dt.to_period('Q')
df_ewn_a = df_ewn_a.rename(columns={'year':'quarter'})
df_ewn_a_q4 = df_ewn_a.sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [41]:
# merge the raw data

df_ewn_a = pd.merge(df_ewn_a_q1, df_ewn_a_q2, how='outer', on=['country', 'quarter', 'ewn_a_extasset', 'ewn_a_extliab', 'ewn_a_gdp'])
df_ewn_a = pd.merge(df_ewn_a, df_ewn_a_q3, how='outer', on=['country', 'quarter', 'ewn_a_extasset', 'ewn_a_extliab', 'ewn_a_gdp'])
df_ewn_a = pd.merge(df_ewn_a, df_ewn_a_q4, how='outer', on=['country', 'quarter', 'ewn_a_extasset', 'ewn_a_extliab', 'ewn_a_gdp'])

In [42]:
# filter

cond1 = (df_ewn_a['country'] == 'Anguilla')
cond2 = (df_ewn_a['country'] == 'Cape Verde')
cond3 = (df_ewn_a['country'] == 'ECCU')
cond4 = (df_ewn_a['country'] == 'Euro Area')
cond5 = (df_ewn_a['country'] == 'Guernsey')
cond6 = (df_ewn_a['country'] == 'Jersey')
cond7 = (df_ewn_a['country'] == 'Montserrat')
cond8 = (df_ewn_a['country'] == 'Netherlands Antilles')
cond9 = (df_ewn_a['country'] == 'Taiwan')

df_ewn_a = df_ewn_a.loc[~cond1 & ~cond2 & ~cond3 & ~cond4 & ~cond5 & ~cond6 & ~cond7 & ~cond8 & ~cond9]

In [43]:
# replace the country

df_ewn_a = df_ewn_a.replace({'country':'Afghanistan, I.R. of'}, 'Afghanistan')
df_ewn_a = df_ewn_a.replace({'country':"C?te d'Ivoire"}, "Cote d'Ivoire")
df_ewn_a = df_ewn_a.replace({'country':'Central African Rep.'}, 'Central African Republic')
df_ewn_a = df_ewn_a.replace({'country':'China,P.R.: Mainland'}, 'China')
df_ewn_a = df_ewn_a.replace({'country':'China,P.R.:Macao'}, 'Macao SAR, China')
df_ewn_a = df_ewn_a.replace({'country':'Congo, Dem. Rep. of'}, 'Congo, Dem. Rep.')
df_ewn_a = df_ewn_a.replace({'country':'Congo, Republic of'}, 'Congo, Rep.')
df_ewn_a = df_ewn_a.replace({'country':'Egypt'}, 'Egypt, Arab Rep.')
df_ewn_a = df_ewn_a.replace({'country':'Hong Kong'}, 'Hong Kong SAR, China')
df_ewn_a = df_ewn_a.replace({'country':'Iran, Islamic Republic of'}, 'Iran, Islamic Rep.')
df_ewn_a = df_ewn_a.replace({'country':'Korea'}, 'Korea, Rep.')
df_ewn_a = df_ewn_a.replace({'country':"Lao People's Dem.Rep"}, 'Lao PDR')
df_ewn_a = df_ewn_a.replace({'country':'Micronesia'}, 'Micronesia, Fed. Sts.')
df_ewn_a = df_ewn_a.replace({'country':'Russia'}, 'Russian Federation')
df_ewn_a = df_ewn_a.replace({'country':'S?o Tom? & Pr?ncipe'}, 'Sao Tome and Principe')
df_ewn_a = df_ewn_a.replace({'country':'Sint Maarten'}, 'Sint Maarten (Dutch part)')
df_ewn_a = df_ewn_a.replace({'country':'St. Vincent & Grens.'}, 'St. Vincent and the Grenadines')
df_ewn_a = df_ewn_a.replace({'country':'Turkey'}, 'Turkiye')
df_ewn_a = df_ewn_a.replace({'country':'Turks and Caicos'}, 'Turks and Caicos Islands')
df_ewn_a = df_ewn_a.replace({'country':'Venezuela, Rep. Bol.'}, 'Venezuela, RB')
df_ewn_a = df_ewn_a.replace({'country':'Yemen, Republic of'}, 'Yemen, Rep.')

In [44]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_ewn_a = pd.merge(df_wb_isocode, df_ewn_a, how='right', on=['country'])
df_ewn_a = df_ewn_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [45]:
# sort the data by year

df_ewn_a['year'] = df_ewn_a['quarter'].dt.year
df_ewn_a = df_ewn_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_ewn_a = df_ewn_a.drop(columns=['year']).reset_index(drop=True)

In [46]:
# select the T and N

# select the T
cond_T = (df_ewn_a['quarter'].dt.year >= 1950)

df_ewn_a = df_ewn_a.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 'CZE',
                          'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN',
                          'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD',
                          'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SVN', 'SWE', 'THA', 'TUR',
                          'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_ewn_a['isocode'] == isocode_array[i])


df_ewn_a = df_ewn_a.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

In [47]:
# annual irr dataset

df_ewn_a['ewn_a_finopen_fa'] = ((df_ewn_a['ewn_a_extasset'] + df_ewn_a['ewn_a_extliab']) / df_ewn_a['ewn_a_gdp']) * 100
df_ewn_a = df_ewn_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_ewn_a

,country,isocode,quarter,ewn_a_extasset,ewn_a_extliab,ewn_a_gdp,ewn_a_finopen_fa
0,Argentina,ARG,1970Q1,1590.35,7675.18,34039.26,27.22
1,Argentina,ARG,1970Q2,1590.35,7675.18,34039.26,27.22
2,Argentina,ARG,1970Q3,1590.35,7675.18,34039.26,27.22
3,Argentina,ARG,1970Q4,1590.35,7675.18,34039.26,27.22
4,Argentina,ARG,1971Q1,1403.06,8145.90,43165.38,22.12
...,...,...,...,...,...,...,...
10395,South Africa,ZAF,2020Q4,543770.36,439136.21,302202.00,325.25
10396,South Africa,ZAF,2021Q1,548729.04,447382.74,418128.06,238.23
10397,South Africa,ZAF,2021Q2,548729.04,447382.74,418128.06,238.23
10398,South Africa,ZAF,2021Q3,548729.04,447382.74,418128.06,238.23


### **4.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|total assets excl. gold|ewn_a_extasset|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|2|total liabilities|ewn_a_extliab|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|3|gross domestic product|ewn_a_gdp|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|4|financial openness, (ext. assets + ext. liabilities) / GDP|ewn_a_finopen_fa|% of GDP|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|

### **4.3. data summary** <br/>

In [48]:
# number of countries & sample period

print('number of countries:', df_ewn_a['isocode'].unique().shape[0])
print('start:', np.min(df_ewn_a['quarter'].unique()))
print('end:', np.max(df_ewn_a['quarter'].unique()))

number of countries: 50
start: 1970Q1
end: 2021Q4


In [49]:
# non-null count by each variables

df_ewn_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10400 entries, 0 to 10399
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype        
---  ------            --------------  -----        
 0   country           10400 non-null  object       
 1   isocode           10400 non-null  object       
 2   quarter           10400 non-null  period[Q-DEC]
 3   ewn_a_extasset    9296 non-null   float64      
 4   ewn_a_extliab     9296 non-null   float64      
 5   ewn_a_gdp         9696 non-null   float64      
 6   ewn_a_finopen_fa  9292 non-null   float64      
dtypes: float64(4), object(2), period[Q-DEC](1)
memory usage: 568.9+ KB


In [50]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_ewn_a.describe(), 2)

,ewn_a_extasset,ewn_a_extliab,ewn_a_gdp,ewn_a_finopen_fa
count,9296.00,9296.00,9696.00,9292.00
mean,1098694.67,1156741.15,640361.13,628.88
std,2969341.53,3417926.59,1778477.87,2831.51
min,299.76,253.97,212.43,9.28
25%,17730.33,36553.79,45700.94,76.70
50%,105631.64,161538.44,161581.47,148.42
75%,640252.27,713458.09,468134.86,304.77
max,34593315.34,53189743.00,22997501.00,41108.32


## **5. IRR**

* [IRR](https://carmenreinhart.com/exchange-rate/#): Ethan Ilzetzki, Carmen M. Reinhart and Kenneth S. Rogoff

### **5.1. python code**

In [51]:
# generate the q1 dataset

var_list = ['irr_a_fx_fine', 'irr_a_fx_coarse']

# q1 function transforming y type into ymd type
def ymd_q1(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-01-01'

# import the raw data
for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv').set_index('year').transpose()
    globals()['df_{}'.format(var)] = pd.DataFrame(globals()['df_{}'.format(var)].stack(level='year')).reset_index()
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'level_0':'country', 0:var})
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)][['country', 'year', var]].sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
for var in var_list:

    globals()['df_{}'.format(var)]['year'] = pd.DataFrame(globals()['df_{}'.format(var)]['year'].apply(ymd_q1))
    globals()['df_{}'.format(var)]['year'] = pd.to_datetime(globals()['df_{}'.format(var)]['year'])
    globals()['df_{}'.format(var)]['year'] = globals()['df_{}'.format(var)]['year'].dt.to_period('Q')
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'year':'quarter'})
    globals()['df_{}_q1'.format(var)] = globals()['df_{}'.format(var)].sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [52]:
# generate the q2 dataset

var_list = ['irr_a_fx_fine', 'irr_a_fx_coarse']

# q2 function transforming y type into ymd type
def ymd_q2(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-04-01'

# import the raw data
for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv').set_index('year').transpose()
    globals()['df_{}'.format(var)] = pd.DataFrame(globals()['df_{}'.format(var)].stack(level='year')).reset_index()
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'level_0':'country', 0:var})
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)][['country', 'year', var]].sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
for var in var_list:

    globals()['df_{}'.format(var)]['year'] = pd.DataFrame(globals()['df_{}'.format(var)]['year'].apply(ymd_q2))
    globals()['df_{}'.format(var)]['year'] = pd.to_datetime(globals()['df_{}'.format(var)]['year'])
    globals()['df_{}'.format(var)]['year'] = globals()['df_{}'.format(var)]['year'].dt.to_period('Q')
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'year':'quarter'})
    globals()['df_{}_q2'.format(var)] = globals()['df_{}'.format(var)].sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [53]:
# generate the q3 dataset

var_list = ['irr_a_fx_fine', 'irr_a_fx_coarse']

# q3 function transforming y type into ymd type
def ymd_q3(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-07-01'

# import the raw data
for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv').set_index('year').transpose()
    globals()['df_{}'.format(var)] = pd.DataFrame(globals()['df_{}'.format(var)].stack(level='year')).reset_index()
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'level_0':'country', 0:var})
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)][['country', 'year', var]].sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
for var in var_list:

    globals()['df_{}'.format(var)]['year'] = pd.DataFrame(globals()['df_{}'.format(var)]['year'].apply(ymd_q3))
    globals()['df_{}'.format(var)]['year'] = pd.to_datetime(globals()['df_{}'.format(var)]['year'])
    globals()['df_{}'.format(var)]['year'] = globals()['df_{}'.format(var)]['year'].dt.to_period('Q')
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'year':'quarter'})
    globals()['df_{}_q3'.format(var)] = globals()['df_{}'.format(var)].sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [54]:
# generate the q4 dataset

var_list = ['irr_a_fx_fine', 'irr_a_fx_coarse']

# q4 function transforming y type into ymd type
def ymd_q4(x):

    # numerical type, not string type -> indexing is not applied
    result = str(x)
    return result[0:4] + '-10-01'

# import the raw data
for var in var_list:

    globals()['df_{}'.format(var)] = pd.read_csv('df_' + var + '.csv').set_index('year').transpose()
    globals()['df_{}'.format(var)] = pd.DataFrame(globals()['df_{}'.format(var)].stack(level='year')).reset_index()
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'level_0':'country', 0:var})
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)][['country', 'year', var]].sort_values(by=['country', 'year']).reset_index(drop=True)

# transform year into quarter
for var in var_list:

    globals()['df_{}'.format(var)]['year'] = pd.DataFrame(globals()['df_{}'.format(var)]['year'].apply(ymd_q4))
    globals()['df_{}'.format(var)]['year'] = pd.to_datetime(globals()['df_{}'.format(var)]['year'])
    globals()['df_{}'.format(var)]['year'] = globals()['df_{}'.format(var)]['year'].dt.to_period('Q')
    globals()['df_{}'.format(var)] = globals()['df_{}'.format(var)].rename(columns={'year':'quarter'})
    globals()['df_{}_q4'.format(var)] = globals()['df_{}'.format(var)].sort_values(by=['country', 'quarter']).reset_index(drop=True)

In [55]:
# merge the raw data

df_irr_a_fx_fine = pd.merge(df_irr_a_fx_fine_q1, df_irr_a_fx_fine_q2, how='outer', on=['country', 'quarter', 'irr_a_fx_fine'])
df_irr_a_fx_fine = pd.merge(df_irr_a_fx_fine, df_irr_a_fx_fine_q3, how='outer', on=['country', 'quarter', 'irr_a_fx_fine'])
df_irr_a_fx_fine = pd.merge(df_irr_a_fx_fine, df_irr_a_fx_fine_q4, how='outer', on=['country', 'quarter', 'irr_a_fx_fine'])

df_irr_a_fx_coarse = pd.merge(df_irr_a_fx_coarse_q1, df_irr_a_fx_coarse_q2, how='outer', on=['country', 'quarter', 'irr_a_fx_coarse'])
df_irr_a_fx_coarse = pd.merge(df_irr_a_fx_coarse, df_irr_a_fx_coarse_q3, how='outer', on=['country', 'quarter', 'irr_a_fx_coarse'])
df_irr_a_fx_coarse = pd.merge(df_irr_a_fx_coarse, df_irr_a_fx_coarse_q4, how='outer', on=['country', 'quarter', 'irr_a_fx_coarse'])

df_irr_a = pd.merge(df_irr_a_fx_fine, df_irr_a_fx_coarse, how='outer', on=['country', 'quarter'])

In [56]:
# merge the wb isocode data

df_wb_isocode = pd.read_csv('df_wb_isocode.csv', encoding='utf-8').rename(columns={'alpha-3':'isocode'})
df_irr_a = pd.merge(df_wb_isocode, df_irr_a, how='right', on=['country'])
df_irr_a = df_irr_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [57]:
# sort the data by year

df_irr_a['year'] = df_irr_a['quarter'].dt.year
df_irr_a = df_irr_a.sort_values(by=['isocode', 'year']).reset_index(drop=True)
df_irr_a = df_irr_a.drop(columns=['year']).reset_index(drop=True)

In [58]:
# select the T and N

# select the T
cond_T = (df_irr_a['quarter'].dt.year >= 1950)

df_irr_a = df_irr_a.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 'CZE',
                          'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN',
                          'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD',
                          'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SVN', 'SWE', 'THA', 'TUR',
                          'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_irr_a['isocode'] == isocode_array[i])


df_irr_a = df_irr_a.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

In [59]:
# annual irr dataset

df_irr_a = df_irr_a.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_irr_a

,country,isocode,quarter,irr_a_fx_fine,irr_a_fx_coarse
0,Argentina,ARG,1950Q1,14.00,5.00
1,Argentina,ARG,1950Q2,14.00,5.00
2,Argentina,ARG,1950Q3,14.00,5.00
3,Argentina,ARG,1950Q4,14.00,5.00
4,Argentina,ARG,1951Q1,14.00,5.00
...,...,...,...,...,...
13395,South Africa,ZAF,2015Q4,12.00,3.00
13396,South Africa,ZAF,2016Q1,12.00,3.00
13397,South Africa,ZAF,2016Q2,12.00,3.00
13398,South Africa,ZAF,2016Q3,12.00,3.00


### **5.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|1|exchange regime classification, fine|irr_a_fx_fine|-|annual|-|-|fine|https://carmenreinhart.com/exchange-rate/#|
|2|exchange regime classification, coarse|irr_a_fx_coarse|-|annual|-|-|coarse|https://carmenreinhart.com/exchange-rate/#|

### **5.3. data summary** <br/>

In [60]:
# number of countries & sample period

print('number of countries:', df_irr_a['isocode'].unique().shape[0])
print('start:', np.min(df_irr_a['quarter'].unique()))
print('end:', np.max(df_irr_a['quarter'].unique()))

number of countries: 50
start: 1950Q1
end: 2016Q4


In [61]:
# non-null count by each variables

df_irr_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13400 entries, 0 to 13399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype        
---  ------           --------------  -----        
 0   country          13400 non-null  object       
 1   isocode          13400 non-null  object       
 2   quarter          13400 non-null  period[Q-DEC]
 3   irr_a_fx_fine    13400 non-null  float64      
 4   irr_a_fx_coarse  13400 non-null  float64      
dtypes: float64(2), object(2), period[Q-DEC](1)
memory usage: 523.6+ KB


In [62]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_irr_a.describe(), 2)

,irr_a_fx_fine,irr_a_fx_coarse
count,13400.00,13400.00
mean,7.23,2.37
std,4.92,1.55
min,1.00,1.00
25%,2.00,1.00
50%,8.00,2.00
75%,12.00,3.00
max,15.00,6.00


## **6. pooled data**

### **6.1. python code** <br/>

In [63]:
# generate the quarterly panel data 2

df_q_panel_2 = pd.merge(df_imf_q, df_wb_a, how='outer', on=['country', 'isocode', 'quarter'])
df_q_panel_2 = pd.merge(df_q_panel_2, df_ci_a, how='outer', on=['country', 'isocode', 'quarter'])
df_q_panel_2 = pd.merge(df_q_panel_2, df_ewn_a, how='outer', on=['country', 'isocode', 'quarter'])
df_q_panel_2 = pd.merge(df_q_panel_2, df_irr_a, how='outer', on=['country', 'isocode', 'quarter'])
df_q_panel_2 = df_q_panel_2.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [64]:
# generate the quarterly temp data

df_q_temp = pd.read_csv('df_q_temp.csv').set_index(['country', 'alpha-3']).transpose()
df_q_temp = pd.DataFrame(df_q_temp.stack(level=['country', 'alpha-3'])).reset_index()
df_q_temp = df_q_temp.rename(columns={'level_0':'quarter', 'alpha-3':'isocode', 0:'value'})
df_q_temp = df_q_temp[['country', 'isocode', 'quarter', 'value']].sort_values(by=['isocode', 'quarter']).reset_index(drop=True)
df_q_temp['quarter'] = pd.to_datetime(df_q_temp['quarter'])
df_q_temp['quarter'] = df_q_temp['quarter'].dt.to_period('Q')

# select the T and N

# select the T
cond_T = (df_q_temp['quarter'].dt.year >= 1950)

df_q_temp = df_q_temp.loc[cond_T]

# select the N
isocode_array = np.array(['ARG', 'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CYP', 'CZE',
                          'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN',
                          'IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD',
                          'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'RUS', 'SGP', 'SVK', 'SVN', 'SWE', 'THA', 'TUR',
                          'USA', 'ZAF'])

for i in range(0, isocode_array.shape[0]):
  
    globals()['cond_N{}'.format(i+1)] = (df_q_temp['isocode'] == isocode_array[i])


df_q_temp = df_q_temp.loc[cond_N1 | cond_N2 | cond_N3 | cond_N4 | cond_N5 | cond_N6 | cond_N7 | cond_N8 | cond_N9 | cond_N10 | cond_N11 | cond_N12 | cond_N13 | cond_N14 | cond_N15 | cond_N16 | cond_N17 | cond_N18 | cond_N19 | cond_N20 | cond_N21 | cond_N22 | cond_N23 | cond_N24 | cond_N25 | cond_N26 | cond_N27 | cond_N28 | cond_N29 | cond_N30 | cond_N31 | cond_N32 | cond_N33 | cond_N34 | cond_N35 | cond_N36 | cond_N37 | cond_N38 | cond_N39 | cond_N40 | cond_N41 | cond_N42 | cond_N43 | cond_N44 | cond_N45 | cond_N46 | cond_N47 | cond_N48 | cond_N49 | cond_N50]

df_q_temp = df_q_temp.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

In [65]:
# merge the quarterly panel data 2 with the quarterly temp data

df_q_panel_2 = pd.merge(df_q_panel_2, df_q_temp, how='outer', on=['country', 'isocode', 'quarter']).drop(columns=['value'])

In [66]:
# quarterly panel 2 dataset

df_q_panel_2 = df_q_panel_2.sort_values(by=['isocode', 'quarter']).reset_index(drop=True)

df_q_panel_2

,country,isocode,quarter,imf_q_ngdp,imf_q_exp,imf_q_imp,imf_q_pcon,imf_q_tradeopen,wb_a_cds_5y,ci_a_kaopen,ci_a_kaopen_n,ewn_a_extasset,ewn_a_extliab,ewn_a_gdp,ewn_a_finopen_fa,irr_a_fx_fine,irr_a_fx_coarse
0,Argentina,ARG,1950Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,5.00
1,Argentina,ARG,1950Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,5.00
2,Argentina,ARG,1950Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,5.00
3,Argentina,ARG,1950Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,5.00
4,Argentina,ARG,1951Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.00,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14495,South Africa,ZAF,2021Q2,1562120000000.00,502886000000.00,377361000000.00,752475000000.00,56.35,211.64,NaN,NaN,548729.04,447382.74,418128.06,238.23,NaN,NaN
14496,South Africa,ZAF,2021Q3,1546850000000.00,472716000000.00,379539000000.00,731776000000.00,55.10,211.64,NaN,NaN,548729.04,447382.74,418128.06,238.23,NaN,NaN
14497,South Africa,ZAF,2021Q4,1571830000000.00,495319000000.00,428015000000.00,753631000000.00,58.74,211.64,NaN,NaN,548729.04,447382.74,418128.06,238.23,NaN,NaN
14498,South Africa,ZAF,2022Q1,1608260000000.00,540074000000.00,466989000000.00,762892000000.00,62.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **6.2. var. summary** <br/>

| |variable|abbreviation|unit|frequency|indicator|subject|measure|source|
|-|--------|------------|----|---------|---------|-------|-------|------|
|$IMF$|||||||||
|1|gross domestic product, nominal|imf_q_ngdp|DCU|quarterly|NGDP_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|2|export of goods and services, nominal|imf_q_exp|DCU|quarterly|NX_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|3|import of goods and services, nominal|imf_q_imp|DCU|quarterly|NM_SA_XDC|-|nominal / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|4|trade openness, (export + import) / GDP|imf_q_tradeopen|% of GDP|quarterly|-|-|-|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|5|private sector final consumption expenditure, real|imf_q_pcon|DCU|quarterly|NCP_R_SA_XDC|-|real / SA|https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1390030341854|
|$WB$|||||||||
|1|5-year sovereign CDS spreads, basis points|wb_a_cds_5y|basis point|annual|-|-|-|https://www.worldbank.org/en/research/brief/fiscal-space|
|$CI$|||||||||
|1|Chinn-Ito financial openness index|ci_a_kaopen|index|annual|-|-|-|https://web.pdx.edu/~ito/Chinn-Ito_website.htm|
|2|Chinn-Ito financial openness index, normalized|ci_a_kaopen_n|index|annual|-|-|normalzied (0-1)|https://web.pdx.edu/~ito/Chinn-Ito_website.htm|
|$EWN$|||||||||
|1|total assets excl. gold|ewn_a_extasset|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|2|total liabilities|ewn_a_extliab|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|3|gross domestic product|ewn_a_gdp|USD|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|4|financial openness, (ext. assets + ext. liabilities) / GDP|ewn_a_finopen_fa|% of GDP|annual|-|-|-|https://www.brookings.edu/research/the-external-wealth-of-nations-database/|
|$IRR$|||||||||
|1|exchange regime classification, fine|irr_a_fx_fine|-|annual|-|-|fine|https://carmenreinhart.com/exchange-rate/#|
|2|exchange regime classification, coarse|irr_a_fx_coarse|-|annual|-|-|coarse|https://carmenreinhart.com/exchange-rate/#|

### **6.3. data summary**

In [67]:
# number of countries & sample period

print('number of countries:', df_q_panel_2['isocode'].unique().shape[0])
print('start:', np.min(df_q_panel_2['quarter'].unique()))
print('end:', np.max(df_q_panel_2['quarter'].unique()))

number of countries: 50
start: 1950Q1
end: 2022Q2


In [68]:
# non-null count by each variables

df_q_panel_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14500 entries, 0 to 14499
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype        
---  ------            --------------  -----        
 0   country           14500 non-null  object       
 1   isocode           14500 non-null  object       
 2   quarter           14500 non-null  period[Q-DEC]
 3   imf_q_ngdp        6191 non-null   float64      
 4   imf_q_exp         6191 non-null   float64      
 5   imf_q_imp         6191 non-null   float64      
 6   imf_q_pcon        6131 non-null   float64      
 7   imf_q_tradeopen   6191 non-null   float64      
 8   wb_a_cds_5y       3204 non-null   float64      
 9   ci_a_kaopen       8600 non-null   float64      
 10  ci_a_kaopen_n     8600 non-null   float64      
 11  ewn_a_extasset    9296 non-null   float64      
 12  ewn_a_extliab     9296 non-null   float64      
 13  ewn_a_gdp         9696 non-null   float64      
 14  ewn_a_finopen_fa  9292 non-null   floa

In [69]:
# descriptive statistics

pd.options.display.float_format = '{:.2f}'.format
round(df_q_panel_2.describe(), 2)

,imf_q_ngdp,imf_q_exp,imf_q_imp,imf_q_pcon,imf_q_tradeopen,wb_a_cds_5y,ci_a_kaopen,ci_a_kaopen_n,ewn_a_extasset,ewn_a_extliab,ewn_a_gdp,ewn_a_finopen_fa,irr_a_fx_fine,irr_a_fx_coarse
count,6191.00,6191.00,6191.00,6131.00,6191.00,3204.00,8600.00,8600.00,9296.00,9296.00,9696.00,9292.00,13400.00,13400.00
mean,42882903097010.86,10665512767181.47,10132811503065.73,23008212211023.31,81.89,318.80,0.76,0.60,1098694.67,1156741.15,640361.13,628.88,7.23,2.37
std,297459619286877.69,66369216310031.29,63027889369226.32,134000145097356.73,55.69,2102.32,1.35,0.49,2969341.53,3417926.59,1778477.87,2831.51,4.92,1.55
min,267985000.00,94170000.00,62187000.00,813515856.40,7.55,1.77,-2.00,0.00,299.76,253.97,212.43,9.28,1.00,1.00
25%,42382768350.50,13782150000.00,14460708446.50,29401150000.00,47.12,34.60,0.00,0.00,17730.33,36553.79,45700.94,76.70,2.00,1.00
50%,204178000000.00,68128500000.00,65993000000.00,153786000000.00,66.17,81.62,1.00,1.00,105631.64,161538.44,161581.47,148.42,8.00,2.00
75%,896335000000.00,292429500000.00,277726000000.00,476171500000.00,102.90,189.79,2.00,1.00,640252.27,713458.09,468134.86,304.77,12.00,3.00
max,4900000000000000.00,1210000000000000.00,1000000000000000.00,1600000000000000.00,395.76,50990.61,2.00,1.00,34593315.34,53189743.00,22997501.00,41108.32,15.00,6.00


## **7. export the data**

In [70]:
# quarterly pooled data

df_q_panel_2.to_excel(excel_writer='df_q_panel_2.xlsx')

## **8. references**

* https://data.imf.org/?sk=4c514d48-b6ba-49ed-8ab9-52b0c1a0179b&sId=1409151240976 <br/>
* https://www.worldbank.org/en/research/brief/fiscal-space <br/>
* https://documents1.worldbank.org/curated/en/601211501678994591/pdf/WPS8157.pdf <br/>
* https://web.pdx.edu/~ito/Chinn-Ito_website.htm <br/>
* https://www.brookings.edu/research/the-external-wealth-of-nations-database/
<br/>
* https://www.imf.org/en/Publications/WP/Issues/2017/05/10/International-Financial-Integration-in-the-Aftermath-of-the-Global-Financial-Crisis-44906 <br/>
* https://carmenreinhart.com/exchange-rate/# <br/>